In [3]:
import pandas as pd
import psycopg2
import numpy as np
import re
from sqlalchemy import create_engine, text as sql_text
import datetime
from ydata_profiling import ProfileReport

# Projeto Analytics Engineering - Inside Airbnb

## PostgreSQL - Exportação dos dados "raw"

Conjunto de dados:  "Inside Airbnb"

Passo a Passo:
1. Vá no link https://insideairbnb.com/get-the-data
2. Procure por "Rio de Janeiro"
3. Baixe os 3 .csv: "Listing", "Reviews" e Calendar" 

0bs: Caso queira pular estes passos, basta utilizar os .csvs que já estão nessa pasta: https://drive.google.com/drive/folders/1pwCpVsMu5si7BX1lzWX3tipP3fAPuoX3?usp=drive_link

Criação do banco de dados PostgreSQL com as 3 tabelas ("Listing", "Reviews" e Calendar"):
1. Rode o arquivo 'create_tables.sql'
2. Rode o arquivo 'copy.sql' alterando o caminho para seus .csvs localmente
3. Crie sua conexão abaixo (não exclua as demais)

In [4]:
# James:
#conn = psycopg2.connect(dbname='airbnb_project', user='postgres', host='localhost', password='123456', port="5432")

# Marcela: 
conn = psycopg2.connect(dbname='myDBProject', user='postgres', host='localhost', password='0125698741', port="5433")
db_params = {
    'dbname': 'myDBProject',
    'user': 'postgres',
    'host': 'localhost',
    'password': '0125698741',
    'port': '5433'
}
db_url = f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}'


In [14]:
engine = create_engine(db_url)
df_listings_raw = pd.read_csv("raw_data/listings.csv")
df_reviews_raw = pd.read_csv("raw_data/reviews.csv")
df_calendar_raw = pd.read_csv("raw_data/calendar.csv")

In [15]:
df_listings_raw.to_sql('listings_raw', engine, if_exists='replace', index=False)
df_reviews_raw.to_sql('reviews_raw', engine, if_exists='replace', index=False)
df_calendar_raw.to_sql('calendar_raw', engine, if_exists='replace', index=False)

595

In [13]:
def load_data_from_table(table_name):
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql_query(query, conn)
    return df
df = load_data_from_table('listings_raw')
df.head(5)

/var/folders/8k/smxjq0q17kz2976yb3t29z6r0000gn/T/ipykernel_51914/846556828.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17878,https://www.airbnb.com/rooms/17878,20231226034138,2023-12-27,city scrape,Condo in Rio de Janeiro · ★4.70 · 2 bedrooms ·...,None,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/pictures/65320518/3069...,68997,...,4.91,4.77,4.67,None,f,1,1,0,0,1.90
1,25026,https://www.airbnb.com/rooms/25026,20231226034138,2023-12-27,city scrape,Rental unit in Rio de Janeiro · ★4.72 · 1 bedr...,None,Copacabana is a lively neighborhood and the ap...,https://a0.muscache.com/pictures/a745aa21-b8dd...,102840,...,4.92,4.84,4.60,None,f,1,1,0,0,1.67
2,35764,https://www.airbnb.com/rooms/35764,20231226034138,2023-12-27,city scrape,Loft in Rio de Janeiro · ★4.90 · 1 bedroom · 1...,None,Our guests will experience living with a local...,https://a0.muscache.com/pictures/23782972/1d3e...,153691,...,4.95,4.94,4.89,None,f,1,1,0,0,2.82
3,41198,https://www.airbnb.com/rooms/41198,20231226034138,2023-12-27,city scrape,Rental unit in Rio de Janeiro · ★4.21 · 2 bedr...,None,None,https://a0.muscache.com/pictures/3576716/2d6a9...,178975,...,4.56,4.44,4.38,None,f,2,2,0,0,0.13
4,326205,https://www.airbnb.com/rooms/326205,20231226034138,2023-12-27,city scrape,Condo in Rio de Janeiro · ★4.57 · 1 bedroom · ...,None,None,https://a0.muscache.com/pictures/c550151d-910c...,1603206,...,4.77,4.83,4.59,None,f,5,5,0,0,1.07


## DBT - Inicialização

Passo a Passo:
1) Instale o DBT
- python -m pip install dbt-postgres
- pip install dbt


2) Vá até sua pasta "dbt" e rode o comando:
3) dbt init analytics_eng_airbnb 

OBS: você pode pular os passos 2 e 3 utilizando a pasta "dbt" dentro do arquivo do Github


# Camadas

## Bronze

Verifique os arquivos criados na pasta dbt/analytics_eng_airbnb/models/bronze
Nessa etapa:
- Criamos as tabelas _bronze para as três  
Verificamos se ID em "listings" é unique e not null
- 

Entre na pasta dbt/analytics_eng_airbnb e rode o comando:

- dbt run --models bronze

## Silver

### Data Quality

- Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
- Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
- Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".


Listing (Listagem): Este conjunto de dados contém informações detalhadas sobre as propriedades listadas no Airbnb. Cada registro representa uma listagem individual e inclui informações como o tipo de propriedade, preço, localização, número de quartos, comodidades oferecidas e muito mais.

Adicionar colunas mantidas e descricao.

Reviews (Avaliações): O conjunto de dados de avaliações contém informações sobre as avaliações feitas por hóspedes que ficaram nas propriedades listadas. Ele inclui dados como a data da avaliação, o identificador da propriedade, os comentários escritos pelos hóspedes, e outras informações.


Adicionar colunas mantidas e descricao.

Calendar (Calendário): Este conjunto de dados contém informações sobre a disponibilidade das propriedades ao longo do tempo. Ele lista as datas em que as propriedades estão disponíveis para reserva, bem como os preços para cada data.


Adicionar colunas mantidas e descricao.

## Testes de qualidade

- Utilize a biblioteca Great Expectations para criar testes de qualidade automatizados que verifiquem as expectativas definidas para os dados da camada "silver".
- Desenvolva testes que assegurem que os dados da camada "silver" atendam às regras de negócios e aos requisitos de qualidade.

## Armazenamento de Dados em PostgreSQL

- Armazene os dados da camada "silver" no mesmo banco de dados PostgreSQL.
- Estabeleça conexões entre o dbt e o PostgreSQL para carregar os dados transformados da camada "silver" no banco.

In [40]:
engine = create_engine(db_url)

In [ ]:
df_listings_silver.to_sql('listings_silver', engine, if_exists='replace', index=False)

In [ ]:
query= """
select * 
from public.listings_gold
"""
df = pd.read_sql(sql=sql_text(query), con=engine.connect())
df

## Validação de Expectativas com Great Expectations

- Implemente validações adicionais usando Great Expectations nas camadas de dados da camada "silver".
- Monitore a qualidade dos dados da camada "silver" após cada transformação e ajuste os testes de acordo.

## Gold